<a href="https://colab.research.google.com/github/Bovi-analytics/SupplementalFiles-YongyanChen/blob/main/4_Descriptive_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparation

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive') 

Mounted at /content/gdrive


In [ ]:
!apt-get update

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [834 kB]
Hit:13 http://ppa.lau

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz
!tar xf spark-3.2.0-bin-hadoop3.2.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop3.2"

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
from pyspark.sql.types import LongType, StringType, StructField, StructType, BooleanType, ArrayType, IntegerType, DoubleType, FloatType
import pyspark
from pyspark.sql import functions as F,Window
from pyspark.sql.functions import year, col, array, monotonically_increasing_id, to_json, struct, round, datediff,unix_timestamp,udf,avg
import json
from urllib.request import  urlopen
import pyspark.sql.functions as F

#Mean(sd) for 3 different methods (Table 1)

## CalvingYear

In [ ]:
CalvingYear = spark. \
  read. \
  parquet('/content/gdrive/Shared drives/Bovi-Analytics/Projects/ChenYongYan/Data/Tableau/HerdSummaryIn3Method1203/CalvingYear') \

In [ ]:
CalvingYear.show(3)

+--------------+-----------+-----------+------------------+------------------+--------------------+--------------------+------------------+-----------------+-------------+----------+------------+-----------+-----------------+------------+--------------+
|HerdIdentifier|ParityGroup|CalvingYear|       MeanScaleKg|          MeanRamp|          MeanOffset|           MeanDecay|   MeanPersistence|       MeanM305Kg|MedianScaleKg|MedianRamp|MedianOffset|MedianDecay|MedianPersistence|MedianM305Kg|NumberOfAnimal|
+--------------+-----------+-----------+------------------+------------------+--------------------+--------------------+------------------+-----------------+-------------+----------+------------+-----------+-----------------+------------+--------------+
|           590|          1|       2015| 39.28324324324324| 29.81351351351351|                -0.5|0.001309189189189...| 660.2972972972973|9296.972972972973|        40.34|      30.0|        -0.5|    0.00133|            522.0|      9302.0|

In [ ]:
CalvingYear.count()

437741

In [ ]:
CalvingYear.\
  filter((col('CalvingYear')>2007)&\
      (col('CalvingYear')<2019)).\
  groupBy('ParityGroup').\
  agg(F.round(F.avg('MeanScaleKg'),2).alias('MeanScale'),
    F.round(F.stddev('MeanScaleKg'),2).alias('MeanScaleSd'),
    F.round(F.expr('percentile(MeanScaleKg, array(0.25))')[0],2).alias('MeanScaleQ1'),
    F.round(F.expr('percentile(MeanScaleKg, array(0.75))')[0],2).alias('MeanScaleQ3'),
    F.round(F.avg('MedianScaleKg'),2).alias('MedianScale'),
    F.round(F.stddev('MedianScaleKg'),2).alias('MedianScaleSd'),
    F.round(F.expr('percentile(MedianScaleKg, array(0.25))')[0],2).alias('MedianScaleQ1'),
    F.round(F.expr('percentile(MedianScaleKg, array(0.75))')[0],2).alias('MedianScaleQ3'),

    F.round(F.avg('MeanRamp'),2).alias('MeanRamp'),
    F.round(F.stddev('MeanRamp'),2).alias('MeanRampSd'),
    F.round(F.expr('percentile(MeanRamp, array(0.25))')[0],2).alias('MeanRampQ1'),
    F.round(F.expr('percentile(MeanRamp, array(0.75))')[0],2).alias('MeanRampQ3'),
    F.round(F.avg('MedianRamp'),2).alias('MedianRamp'),
    F.round(F.stddev('MedianRamp'),2).alias('MedianRampSd'),
    F.round(F.expr('percentile(MedianRamp, array(0.25))')[0],2).alias('MedianRampQ1'),
    F.round(F.expr('percentile(MedianRamp, array(0.75))')[0],2).alias('MedianRampQ3'),


    F.round(F.avg('MeanPersistence'),2).alias('MeanPersistence'),
    F.round(F.stddev('MeanPersistence'),2).alias('MeanPersistenceSd'),
    F.round(F.expr('percentile(MeanPersistence, array(0.25))')[0],2).alias('MeanPersistenceQ1'),
    F.round(F.expr('percentile(MeanPersistence, array(0.75))')[0],2).alias('MeanPersistenceQ3'),
    F.round(F.avg('MedianPersistence'),2).alias('MedianPersistence'),
    F.round(F.stddev('MedianPersistence'),2).alias('MedianPersistenceSd'),
    F.round(F.expr('percentile(MedianPersistence, array(0.25))')[0],2).alias('MedianPersistenceQ1'),
    F.round(F.expr('percentile(MedianPersistence, array(0.75))')[0],2).alias('MedianPersistenceQ3')
    ).\
    orderBy('ParityGroup').show()

+-----------+---------+-----------+-----------+-----------+-----------+-------------+-------------+-------------+--------+----------+----------+----------+----------+------------+------------+------------+---------------+-----------------+-----------------+-----------------+-----------------+-------------------+-------------------+-------------------+
|ParityGroup|MeanScale|MeanScaleSd|MeanScaleQ1|MeanScaleQ3|MedianScale|MedianScaleSd|MedianScaleQ1|MedianScaleQ3|MeanRamp|MeanRampSd|MeanRampQ1|MeanRampQ3|MedianRamp|MedianRampSd|MedianRampQ1|MedianRampQ3|MeanPersistence|MeanPersistenceSd|MeanPersistenceQ1|MeanPersistenceQ3|MedianPersistence|MedianPersistenceSd|MedianPersistenceQ1|MedianPersistenceQ3|
+-----------+---------+-----------+-----------+-----------+-----------+-------------+-------------+-------------+--------+----------+----------+----------+----------+------------+------------+------------+---------------+-----------------+-----------------+-----------------+-----------------

## EndYear

In [ ]:
EndYear = spark. \
  read. \
  parquet('/content/gdrive/Shared drives/Bovi-Analytics/Projects/ChenYongYan/Data/Tableau/HerdSummaryIn3Method1203/EndYear') \

In [ ]:
EndYear.show(3)

+--------------+-----------+-------+-----------------+------------------+--------------------+--------------------+-----------------+-----------------+-------------+----------+------------+--------------------+-----------------+------------+--------------+
|HerdIdentifier|ParityGroup|EndYear|      MeanScaleKg|          MeanRamp|          MeanOffset|           MeanDecay|  MeanPersistence|       MeanM305Kg|MedianScaleKg|MedianRamp|MedianOffset|         MedianDecay|MedianPersistence|MedianM305Kg|NumberOfAnimal|
+--------------+-----------+-------+-----------------+------------------+--------------------+--------------------+-----------------+-----------------+-------------+----------+------------+--------------------+-----------------+------------+--------------+
|           590|          1|   2015|41.17617647058824|29.732352941176472|                -0.5|0.001485294117647...|542.3823529411765|9511.411764705883|        42.76|      29.9|        -0.5|            0.001495|            464.0| 

In [ ]:
EndYear.count()

380309

In [ ]:
EndYear.\
  filter((col('EndYear')>2007)&\
      (col('EndYear')<2019)).\
  groupBy('ParityGroup').\
  agg(F.round(F.avg('MeanScaleKg'),2).alias('MeanScale'),
    F.round(F.stddev('MeanScaleKg'),2).alias('MeanScaleSd'),
    F.round(F.expr('percentile(MeanScaleKg, array(0.25))')[0],2).alias('MeanScaleQ1'),
    F.round(F.expr('percentile(MeanScaleKg, array(0.75))')[0],2).alias('MeanScaleQ3'),
    F.round(F.avg('MedianScaleKg'),2).alias('MedianScale'),
    F.round(F.stddev('MedianScaleKg'),2).alias('MedianScaleSd'),
    F.round(F.expr('percentile(MedianScaleKg, array(0.25))')[0],2).alias('MedianScaleQ1'),
    F.round(F.expr('percentile(MedianScaleKg, array(0.75))')[0],2).alias('MedianScaleQ3'),

    F.round(F.avg('MeanRamp'),2).alias('MeanRamp'),
    F.round(F.stddev('MeanRamp'),2).alias('MeanRampSd'),
    F.round(F.expr('percentile(MeanRamp, array(0.25))')[0],2).alias('MeanRampQ1'),
    F.round(F.expr('percentile(MeanRamp, array(0.75))')[0],2).alias('MeanRampQ3'),
    F.round(F.avg('MedianRamp'),2).alias('MedianRamp'),
    F.round(F.stddev('MedianRamp'),2).alias('MedianRampSd'),
    F.round(F.expr('percentile(MedianRamp, array(0.25))')[0],2).alias('MedianRampQ1'),
    F.round(F.expr('percentile(MedianRamp, array(0.75))')[0],2).alias('MedianRampQ3'),


    F.round(F.avg('MeanPersistence'),2).alias('MeanPersistence'),
    F.round(F.stddev('MeanPersistence'),2).alias('MeanPersistenceSd'),
    F.round(F.expr('percentile(MeanPersistence, array(0.25))')[0],2).alias('MeanPersistenceQ1'),
    F.round(F.expr('percentile(MeanPersistence, array(0.75))')[0],2).alias('MeanPersistenceQ3'),
    F.round(F.avg('MedianPersistence'),2).alias('MedianPersistence'),
    F.round(F.stddev('MedianPersistence'),2).alias('MedianPersistenceSd'),
    F.round(F.expr('percentile(MedianPersistence, array(0.25))')[0],2).alias('MedianPersistenceQ1'),
    F.round(F.expr('percentile(MedianPersistence, array(0.75))')[0],2).alias('MedianPersistenceQ3')
    ).\
    orderBy('ParityGroup').show()

+-----------+---------+-----------+-----------+-----------+-----------+-------------+-------------+-------------+--------+----------+----------+----------+----------+------------+------------+------------+---------------+-----------------+-----------------+-----------------+-----------------+-------------------+-------------------+-------------------+
|ParityGroup|MeanScale|MeanScaleSd|MeanScaleQ1|MeanScaleQ3|MedianScale|MedianScaleSd|MedianScaleQ1|MedianScaleQ3|MeanRamp|MeanRampSd|MeanRampQ1|MeanRampQ3|MedianRamp|MedianRampSd|MedianRampQ1|MedianRampQ3|MeanPersistence|MeanPersistenceSd|MeanPersistenceQ1|MeanPersistenceQ3|MedianPersistence|MedianPersistenceSd|MedianPersistenceQ1|MedianPersistenceQ3|
+-----------+---------+-----------+-----------+-----------+-----------+-------------+-------------+-------------+--------+----------+----------+----------+----------+------------+------------+------------+---------------+-----------------+-----------------+-----------------+-----------------

##Weighted

In [ ]:
Weighted = spark. \
  read. \
  parquet('/content/gdrive/Shared drives/Bovi-Analytics/Projects/ChenYongYan/Data/Tableau/HerdSummaryIn3Method1203/Weight/Weighted') \

In [ ]:
Weighted.show(3)

+--------------+--------+-----------+-----------------+------------------+-------------------+--------------------+------------------+------------------+--------------+-----------+-------------+----------+------------+-----------+-----------------+------------+
|HerdIdentifier|TestYear|ParityGroup|      MeanScaleKg|          MeanRamp|         MeanOffset|           MeanDecay|   MeanPersistence|        MeanM305Kg|NumberOfAnimal|TotalWeight|MedianScaleKg|MedianRamp|MedianOffset|MedianDecay|MedianPersistence|MedianM305Kg|
+--------------+--------+-----------+-----------------+------------------+-------------------+--------------------+------------------+------------------+--------------+-----------+-------------+----------+------------+-----------+-----------------+------------+
|        311983|    2017|         2+|56.44454277286139|27.140117994100294|-0.2702064896755163|0.002843447474022...|286.24631268436576|10785.066135434263|           276|       1356|        55.83|      23.6|         

In [ ]:
Weighted.count()

380607

In [ ]:
Weighted.\
  filter((col('TestYear')>2007)&\
      (col('TestYear')<2019)).\
  groupBy('ParityGroup').\
  agg(F.round(F.avg('MeanScaleKg'),2).alias('MeanScale'),
    F.round(F.stddev('MeanScaleKg'),2).alias('MeanScaleSd'),
    F.round(F.expr('percentile(MeanScaleKg, array(0.25))')[0],2).alias('MeanScaleQ1'),
    F.round(F.expr('percentile(MeanScaleKg, array(0.75))')[0],2).alias('MeanScaleQ3'),
    F.round(F.avg('MedianScaleKg'),2).alias('MedianScale'),
    F.round(F.stddev('MedianScaleKg'),2).alias('MedianScaleSd'),
    F.round(F.expr('percentile(MedianScaleKg, array(0.25))')[0],2).alias('MedianScaleQ1'),
    F.round(F.expr('percentile(MedianScaleKg, array(0.75))')[0],2).alias('MedianScaleQ3'),

    F.round(F.avg('MeanRamp'),2).alias('MeanRamp'),
    F.round(F.stddev('MeanRamp'),2).alias('MeanRampSd'),
    F.round(F.expr('percentile(MeanRamp, array(0.25))')[0],2).alias('MeanRampQ1'),
    F.round(F.expr('percentile(MeanRamp, array(0.75))')[0],2).alias('MeanRampQ3'),
    F.round(F.avg('MedianRamp'),2).alias('MedianRamp'),
    F.round(F.stddev('MedianRamp'),2).alias('MedianRampSd'),
    F.round(F.expr('percentile(MedianRamp, array(0.25))')[0],2).alias('MedianRampQ1'),
    F.round(F.expr('percentile(MedianRamp, array(0.75))')[0],2).alias('MedianRampQ3'),


    F.round(F.avg('MeanPersistence'),2).alias('MeanPersistence'),
    F.round(F.stddev('MeanPersistence'),2).alias('MeanPersistenceSd'),
    F.round(F.expr('percentile(MeanPersistence, array(0.25))')[0],2).alias('MeanPersistenceQ1'),
    F.round(F.expr('percentile(MeanPersistence, array(0.75))')[0],2).alias('MeanPersistenceQ3'),
    F.round(F.avg('MedianPersistence'),2).alias('MedianPersistence'),
    F.round(F.stddev('MedianPersistence'),2).alias('MedianPersistenceSd'),
    F.round(F.expr('percentile(MedianPersistence, array(0.25))')[0],2).alias('MedianPersistenceQ1'),
    F.round(F.expr('percentile(MedianPersistence, array(0.75))')[0],2).alias('MedianPersistenceQ3')

    ).\
    orderBy('ParityGroup').show()

+-----------+---------+-----------+-----------+-----------+-----------+-------------+-------------+-------------+--------+----------+----------+----------+----------+------------+------------+------------+---------------+-----------------+-----------------+-----------------+-----------------+-------------------+-------------------+-------------------+
|ParityGroup|MeanScale|MeanScaleSd|MeanScaleQ1|MeanScaleQ3|MedianScale|MedianScaleSd|MedianScaleQ1|MedianScaleQ3|MeanRamp|MeanRampSd|MeanRampQ1|MeanRampQ3|MedianRamp|MedianRampSd|MedianRampQ1|MedianRampQ3|MeanPersistence|MeanPersistenceSd|MeanPersistenceQ1|MeanPersistenceQ3|MedianPersistence|MedianPersistenceSd|MedianPersistenceQ1|MedianPersistenceQ3|
+-----------+---------+-----------+-----------+-----------+-----------+-------------+-------------+-------------+--------+----------+----------+----------+----------+------------+------------+------------+---------------+-----------------+-----------------+-----------------+-----------------

# Evolution trends

## Calving year

In [ ]:
CalvingYearPerParityGroup = spark. \
  read. \
  parquet('/content/gdrive/Shared drives/Bovi-Analytics/Projects/ChenYongYan/Data/Tableau/HerdSummaryIn3Method1203/CalvingYear') \

In [ ]:
CalvingYearPerParityGroup.count()

437741

In [ ]:
CalvingYearPerParityGroup.show(3)

+--------------+-----------+-----------+------------------+------------------+--------------------+--------------------+-----------------+------------------+-------------+----------+------------+--------------------+-----------------+------------+--------------+
|HerdIdentifier|ParityGroup|CalvingYear|       MeanScaleKg|          MeanRamp|          MeanOffset|           MeanDecay|  MeanPersistence|        MeanM305Kg|MedianScaleKg|MedianRamp|MedianOffset|         MedianDecay|MedianPersistence|MedianM305Kg|NumberOfAnimal|
+--------------+-----------+-----------+------------------+------------------+--------------------+--------------------+-----------------+------------------+-------------+----------+------------+--------------------+-----------------+------------+--------------+
|           590|          1|       2015|40.052121212121214|29.745454545454542|                -0.5|0.001411515151515...|558.1818181818181| 9350.818181818182|        40.98|      30.0|        -0.5|             0.0

In [ ]:
CalvingYearPerParityGroup.describe(['CalvingYear']).show()

+-------+------------------+
|summary|       CalvingYear|
+-------+------------------+
|  count|            437741|
|   mean|2011.8596316086453|
| stddev| 4.181104289471681|
|    min|              2005|
|    max|              2019|
+-------+------------------+



In [ ]:
CalvingYearPerParityGroup.\
  filter((col('CalvingYear')>2007)&\
      (col('CalvingYear')<2019)).\
  groupby('ParityGroup','CalvingYear').\
  agg(F.countDistinct('HerdIdentifier').alias('NumberOfHerd'),
    F.avg('MeanScaleKg').alias('AverageMeanScale'),
    F.stddev('MeanScaleKg').alias('SDMeanScale'),
    F.avg('MedianScaleKg').alias('AverageMedianScale'),
    F.stddev('MedianScaleKg').alias('SDMedianScale'),
    F.avg('MeanRamp').alias('AverageMeanRamp'),
    F.stddev('MeanRamp').alias('SDMeanRamp'),
    F.avg('MedianRamp').alias('AverageMedianRamp'),
    F.stddev('MedianRamp').alias('SDMedianRamp'),
    F.avg('MeanPersistence').alias('AverageMeanPersistence'),
    F.stddev('MeanPersistence').alias('SDMeanPersistence'),
    F.avg('MedianPersistence').alias('AverageMedianPersistence'),
    F.stddev('MedianPersistence').alias('SDMedianPersistence')).\
  orderBy('ParityGroup','CalvingYear').\
  show(30)

+-----------+-----------+------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+-------------------+----------------------+------------------+------------------------+-------------------+
|ParityGroup|CalvingYear|NumberOfHerd|  AverageMeanScale|       SDMeanScale|AverageMedianScale|     SDMedianScale|   AverageMeanRamp|         SDMeanRamp| AverageMedianRamp|       SDMedianRamp|AverageMeanPersistence| SDMeanPersistence|AverageMedianPersistence|SDMedianPersistence|
+-----------+-----------+------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+-------------------+----------------------+------------------+------------------------+-------------------+
|          1|       2008|       16442| 34.11908250473207|3.9187511738990097|  33.9853357255805| 4.122953977556861|29.517986913093566|0.48428763593533586|29.6334

## End year

In [ ]:
EndYearPerParityGroup = spark. \
  read. \
  parquet('/content/gdrive/Shared drives/Bovi-Analytics/Projects/ChenYongYan/Data/Tableau/HerdSummaryIn3Method1203/EndYear')

In [ ]:
EndYearPerParityGroup.count()

380309

In [ ]:
EndYearPerParityGroup.show(3)

+--------------+-----------+-------+-----------------+------------------+-------------------+--------------------+-----------------+------------------+-------------+----------+------------+-----------+-----------------+------------+--------------+
|HerdIdentifier|ParityGroup|EndYear|      MeanScaleKg|          MeanRamp|         MeanOffset|           MeanDecay|  MeanPersistence|        MeanM305Kg|MedianScaleKg|MedianRamp|MedianOffset|MedianDecay|MedianPersistence|MedianM305Kg|NumberOfAnimal|
+--------------+-----------+-------+-----------------+------------------+-------------------+--------------------+-----------------+------------------+-------------+----------+------------+-----------+-----------------+------------+--------------+
|           590|          1|   2015|41.17617647058824|29.732352941176472|               -0.5|0.001485294117647...|542.3823529411765| 9511.411764705883|        42.76|      29.9|        -0.5|   0.001495|            464.0|      9661.5|            34|
|       

In [ ]:
EndYearPerParityGroup.describe(['EndYear']).show()

+-------+------------------+
|summary|           EndYear|
+-------+------------------+
|  count|            380309|
|   mean|2012.7280632327922|
| stddev|3.7198438104729363|
|    min|              2007|
|    max|              2019|
+-------+------------------+



In [ ]:
EndYearPerParityGroup.\
  filter((col('EndYear')>2007)&\
      (col('EndYear')<2019)).\
  groupby('ParityGroup','EndYear').\
  agg(F.countDistinct('HerdIdentifier').alias('NumberOfHerd'),
    F.avg('MeanScaleKg').alias('AverageMeanScale'),
    F.stddev('MeanScaleKg').alias('SDMeanScale'),
    F.avg('MedianScaleKg').alias('AverageMedianScale'),
    F.stddev('MedianScaleKg').alias('SDMedianScale'),
    F.avg('MeanRamp').alias('AverageMeanRamp'),
    F.stddev('MeanRamp').alias('SDMeanRamp'),
    F.avg('MedianRamp').alias('AverageMedianRamp'),
    F.stddev('MedianRamp').alias('SDMedianRamp'),
    F.avg('MeanPersistence').alias('AverageMeanPersistence'),
    F.stddev('MeanPersistence').alias('SDMeanPersistence'),
    F.avg('MedianPersistence').alias('AverageMedianPersistence'),
    F.stddev('MedianPersistence').alias('SDMedianPersistence')).\
  orderBy('ParityGroup','EndYear').\
  show(30)

+-----------+-------+------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+-------------------+----------------------+------------------+------------------------+-------------------+
|ParityGroup|EndYear|NumberOfHerd|  AverageMeanScale|       SDMeanScale|AverageMedianScale|     SDMedianScale|   AverageMeanRamp|         SDMeanRamp| AverageMedianRamp|       SDMedianRamp|AverageMeanPersistence| SDMeanPersistence|AverageMedianPersistence|SDMedianPersistence|
+-----------+-------+------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+-------------------+----------------------+------------------+------------------------+-------------------+
|          1|   2008|       16181| 33.89999896757814|3.8438095698400243| 33.76588375254892|4.0533265006396055| 29.48098332900941| 0.4974323357176201|29.621821889870812| 0.4

## Weighted

In [ ]:
WeightedPerParityGroup = spark. \
  read. \
  parquet('/content/gdrive/Shared drives/Bovi-Analytics/Projects/ChenYongYan/Data/Tableau/HerdSummaryIn3Method1203/Weight/Weighted')

In [ ]:
WeightedPerParityGroup.show(3)

+--------------+--------+-----------+-----------------+------------------+--------------------+--------------------+------------------+------------------+--------------+-----------+-------------+----------+------------+-----------+-----------------+------------+
|HerdIdentifier|TestYear|ParityGroup|      MeanScaleKg|          MeanRamp|          MeanOffset|           MeanDecay|   MeanPersistence|        MeanM305Kg|NumberOfAnimal|TotalWeight|MedianScaleKg|MedianRamp|MedianOffset|MedianDecay|MedianPersistence|MedianM305Kg|
+--------------+--------+-----------+-----------------+------------------+--------------------+--------------------+------------------+------------------+--------------+-----------+-------------+----------+------------+-----------+-----------------+------------+
|        311983|    2017|         2+|55.46420924574209|25.840551500405518|-0.29261962692619636|0.002771730489613701|282.17761557177613|10747.456153432668|           263|       1233|        55.48|      23.3|     

In [ ]:
WeightedPerParityGroup.count()

380607

In [ ]:
WeightedPerParityGroup.\
  filter((col('TestYear')>2007)&\
      (col('TestYear')<2019)).\
  groupby('ParityGroup','TestYear').\
  agg(F.countDistinct('HerdIdentifier').alias('NumberOfHerd'),
    F.avg('MeanScaleKg').alias('AverageMeanScale'),
    F.stddev('MeanScaleKg').alias('SDMeanScale'),
    F.avg('MedianScaleKg').alias('AverageMedianScale'),
    F.stddev('MedianScaleKg').alias('SDMedianScale'),
    F.avg('MeanRamp').alias('AverageMeanRamp'),
    F.stddev('MeanRamp').alias('SDMeanRamp'),
    F.avg('MedianRamp').alias('AverageMedianRamp'),
    F.stddev('MedianRamp').alias('SDMedianRamp'),
    F.avg('MeanPersistence').alias('AverageMeanPersistence'),
    F.stddev('MeanPersistence').alias('SDMeanPersistence'),
    F.avg('MedianPersistence').alias('AverageMedianPersistence'),
    F.stddev('MedianPersistence').alias('SDMedianPersistence')).\
  orderBy('ParityGroup','TestYear').\
  show(30)

+-----------+--------+------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+-------------------+----------------------+------------------+------------------------+-------------------+
|ParityGroup|TestYear|NumberOfHerd|  AverageMeanScale|       SDMeanScale|AverageMedianScale|     SDMedianScale|   AverageMeanRamp|         SDMeanRamp| AverageMedianRamp|       SDMedianRamp|AverageMeanPersistence| SDMeanPersistence|AverageMedianPersistence|SDMedianPersistence|
+-----------+--------+------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+-------------------+----------------------+------------------+------------------------+-------------------+
|          1|    2008|       16207| 33.94067804799886| 3.805547539291989|33.816106620595576|4.0061891224938675|29.431552002876664| 0.4898839794877628|29.572619855618026|